In [67]:
import pandas as pd
import nltk
import numpy as np
from torchtext.data import Field
import csv
from torchtext.data import TabularDataset
from torchtext.data import Iterator, BucketIterator

In [68]:
tokenize = lambda x: x.split()
TEXT = Field(sequential=True, tokenize=tokenize, lower=True, fix_length=200)
LABEL = Field(sequential=False, use_vocab=False)

In [69]:
train_stances = pd.read_csv("fn_data/train_stances.csv")
train_bodies = pd.read_csv("fn_data/train_bodies.csv")

In [70]:
train = pd.read_csv("lstmstuff/train_data_consolidated.csv")
val = pd.read_csv("lstmstuff/val_data_consolidated.csv")

In [71]:
train_datafields = [("Body ID", None),
                 ("Headline", TEXT),
                    ("Body", TEXT),
                 ("unrelated", LABEL),
                ("related", LABEL)]

trn, vld = TabularDataset.splits(
               path="",
               train='lstmstuff/train_data_consolidated.csv', validation="lstmstuff/val_data_consolidated.csv",
               format='csv',
               skip_header=True,
               fields=train_datafields)

In [72]:
TEXT.build_vocab(trn, vectors = 'glove.6B.100d')

In [73]:
train_iter, val_iter = BucketIterator.splits(
 (trn, vld),
 batch_sizes=(64,64),
 sort_key=lambda x: len(x.Body), # the BucketIterator needs to be told what function it should use to group the data.
 sort_within_batch=False,
 repeat=False
)

In [74]:
import torch
class BatchGenerator:
    def __init__(self, dl, x_field1, x_field2, y_field):
        self.dl, self.x_field1, self.x_field2, self.y_field = dl, x_field1, x_field2, y_field
        
    def __len__(self):
        return len(self.dl)
    
    def __iter__(self):
        for batch in self.dl:
            X1 = getattr(batch, self.x_field1)
            X2 = getattr(batch, self.x_field2)
            y = getattr(batch, self.y_field)
            yield (X1, X2, y)
            
train_batch_it = BatchGenerator(train_iter, 'Body','Headline', 'unrelated')
val_batch_it = BatchGenerator(val_iter, 'Body', 'Headline', 'unrelated')

In [75]:
import torch.nn as nn
import torch.nn.functional as F
# class RNN(nn.Module):
#     def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
#         super().__init__()
        
#         self.embedding = nn.Embedding(input_dim, embedding_dim)
#         self.lstm_headline = nn.LSTM(embedding_dim, hidden_dim)
#         self.lstm_article = nn.LSTM(embedding_dim, hidden_dim)
#         self.fc = nn.Linear(hidden_dim, output_dim)
        
#     def forward(self, x):

#         #x = [sent len, batch size]
        
#         embedded = self.embedding(x)
        
#         #embedded = [sent len, batch size, emb dim]
        
#         output, hidden = self.rnn(embedded)
        
#         #output = [sent len, batch size, hid dim]
#         #hidden = [1, batch size, hid dim]
        
#         assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
#         return self.fc(hidden.squeeze(0))

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
class LSTMClassifier(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, output_dim, batch_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm_headline = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
        self.lstm_article = nn.LSTM(hidden_dim, hidden_dim, bidirectional=True)
        self.fc = nn.Linear(hidden_dim*2, output_dim*8)
        self.fc2 = nn.Linear(output_dim*8, output_dim)
        self.batch_size = batch_size
        self.hidden1 = self.init_hidden()
        self.hidden2 = self.init_hidden()
        
    def init_hidden(self):
        h0 = Variable(torch.zeros(2, self.batch_size, self.hidden_dim))
        c0 = Variable(torch.zeros(2, self.batch_size, self.hidden_dim))
        return (h0, c0)
    
    def forward(self, headline, article):
        headline_emb = self.embed(headline)
        
        article_emb = self.embed(article)
        
        lstm_headline_out, hidden = self.lstm_headline(headline_emb, self.hidden)
        
        sum_padded_out_lstm1 = 0
        for tensor in torch.split(lstm_headline_out, self.hidden_dim, dim=2):
            sum_padded_out_lstm1 += tensor

        lstm_article_out, hidden2 = self.lstm_article(sum_padded_out_lstm1, self.hidden2)
        res = self.fc(torch.cat((lstm_article_out[-1, :, :self.hidden_dim], lstm_article_out[0, :, self.hidden_dim:]), 1))
        res2 = self.fc2(F.relu(res))
        return res2
        

In [76]:
# model = RNN(len(TEXT.vocab), 100, 256, 1)
model = LSTMClassifier(100, 8, len(TEXT.vocab), 1, 64)

In [77]:
inputs = torch.randn(3, requires_grad=True)
targets = torch.empty(3).random_(2)
print(inputs)
print(targets)

tensor([ 0.3296, -0.0837, -1.2117], requires_grad=True)
tensor([1., 0., 0.])


In [78]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

In [79]:
counter = []
loss_history = [] 
iteration_number= 0

model.train()
# Train the model
for epoch in range(0,2):
    for i, data in enumerate(train_batch_it,0):
        img0, img1 , label = data
        print("inputs are")
        print(img0.shape)
        print(img1.shape)
        optimizer.zero_grad()
        model.hidden = model.init_hidden()
        model.hidden2 = model.init_hidden()
        output = model(img0,img1)
        print("OUTPUT")
        print(output)
        print(label)
        loss = criterion(output.squeeze(),label.squeeze().float())
        loss.backward()
        optimizer.step()
        if True:
            print("Epoch number {}\n Current loss {}\n".format(epoch,loss.item()))
            iteration_number +=10
            counter.append(iteration_number)
            loss_history.append(loss.item())


inputs are
torch.Size([200, 64])
torch.Size([200, 64])
OUTPUT
tensor([[0.3814],
        [0.3800],
        [0.3698],
        [0.3796],
        [0.3918],
        [0.3804],
        [0.3710],
        [0.3951],
        [0.4026],
        [0.3450],
        [0.3763],
        [0.3923],
        [0.3922],
        [0.3920],
        [0.4257],
        [0.3836],
        [0.3957],
        [0.3657],
        [0.3450],
        [0.3889],
        [0.3698],
        [0.3651],
        [0.3817],
        [0.4167],
        [0.3957],
        [0.3577],
        [0.3750],
        [0.3612],
        [0.4201],
        [0.3667],
        [0.3472],
        [0.3711],
        [0.4016],
        [0.3683],
        [0.3471],
        [0.3969],
        [0.4102],
        [0.3748],
        [0.3601],
        [0.3906],
        [0.3753],
        [0.3796],
        [0.3829],
        [0.4004],
        [0.3865],
        [0.3845],
        [0.3654],
        [0.3651],
        [0.3742],
        [0.4009],
        [0.3657],
        [0.3819],
  

Epoch number 0
 Current loss 0.6360768675804138

inputs are
torch.Size([200, 64])
torch.Size([200, 64])
OUTPUT
tensor([[0.4290],
        [0.4600],
        [0.4259],
        [0.4178],
        [0.4262],
        [0.4622],
        [0.4087],
        [0.4302],
        [0.4121],
        [0.4548],
        [0.4318],
        [0.4298],
        [0.4123],
        [0.4096],
        [0.4153],
        [0.4386],
        [0.4280],
        [0.4654],
        [0.4487],
        [0.4293],
        [0.4016],
        [0.4368],
        [0.4534],
        [0.4290],
        [0.4065],
        [0.4334],
        [0.4244],
        [0.4344],
        [0.4405],
        [0.4254],
        [0.4153],
        [0.4517],
        [0.4262],
        [0.4188],
        [0.4147],
        [0.4480],
        [0.4262],
        [0.4218],
        [0.4709],
        [0.4142],
        [0.4483],
        [0.4159],
        [0.4257],
        [0.3814],
        [0.4125],
        [0.3957],
        [0.4187],
        [0.4242],
        [0.4508],
       

Epoch number 0
 Current loss 0.608489453792572

inputs are
torch.Size([200, 64])
torch.Size([200, 64])
OUTPUT
tensor([[0.4616],
        [0.4889],
        [0.4963],
        [0.4943],
        [0.5140],
        [0.4836],
        [0.4940],
        [0.4959],
        [0.4743],
        [0.4601],
        [0.4815],
        [0.4666],
        [0.4669],
        [0.4156],
        [0.4658],
        [0.4863],
        [0.4578],
        [0.4720],
        [0.4571],
        [0.4883],
        [0.4944],
        [0.4597],
        [0.4921],
        [0.4726],
        [0.4968],
        [0.5092],
        [0.4566],
        [0.4628],
        [0.4447],
        [0.4940],
        [0.4899],
        [0.4886],
        [0.4932],
        [0.4754],
        [0.5187],
        [0.4677],
        [0.4700],
        [0.5045],
        [0.4614],
        [0.5165],
        [0.5035],
        [0.4446],
        [0.4986],
        [0.5004],
        [0.4797],
        [0.4648],
        [0.4183],
        [0.4858],
        [0.4711],
        

Epoch number 0
 Current loss 0.5757606625556946

inputs are
torch.Size([200, 64])
torch.Size([200, 64])
OUTPUT
tensor([[0.4904],
        [0.5300],
        [0.5064],
        [0.5428],
        [0.5794],
        [0.5211],
        [0.5535],
        [0.4943],
        [0.5405],
        [0.5018],
        [0.5464],
        [0.5311],
        [0.5573],
        [0.4633],
        [0.5303],
        [0.5210],
        [0.5465],
        [0.5101],
        [0.5094],
        [0.5316],
        [0.5486],
        [0.4985],
        [0.5156],
        [0.4884],
        [0.4884],
        [0.5288],
        [0.5013],
        [0.5660],
        [0.4884],
        [0.5486],
        [0.4997],
        [0.5181],
        [0.5530],
        [0.5164],
        [0.5288],
        [0.5448],
        [0.4811],
        [0.4915],
        [0.5248],
        [0.4937],
        [0.5012],
        [0.5185],
        [0.5740],
        [0.5446],
        [0.5157],
        [0.5443],
        [0.4825],
        [0.5333],
        [0.5415],
       

Epoch number 0
 Current loss 0.616249680519104

inputs are
torch.Size([200, 64])
torch.Size([200, 64])
OUTPUT
tensor([[0.5605],
        [0.5888],
        [0.5411],
        [0.5549],
        [0.5677],
        [0.5723],
        [0.5781],
        [0.5930],
        [0.5501],
        [0.5985],
        [0.5469],
        [0.5922],
        [0.5885],
        [0.6116],
        [0.5283],
        [0.5783],
        [0.6145],
        [0.6282],
        [0.5306],
        [0.6421],
        [0.6393],
        [0.6310],
        [0.5958],
        [0.5724],
        [0.6146],
        [0.6313],
        [0.5620],
        [0.5306],
        [0.5742],
        [0.5515],
        [0.5806],
        [0.5416],
        [0.5587],
        [0.5901],
        [0.6465],
        [0.5555],
        [0.5659],
        [0.5952],
        [0.5802],
        [0.5624],
        [0.6150],
        [0.5430],
        [0.5552],
        [0.5703],
        [0.5523],
        [0.6024],
        [0.5465],
        [0.6168],
        [0.5858],
        

...

Epoch number 0
 Current loss 0.6256895065307617

inputs are
torch.Size([200, 64])
torch.Size([200, 64])
OUTPUT
tensor([[0.9387],
        [1.1265],
        [0.6920],
        [0.9701],
        [0.8686],
        [1.0343],
        [1.0133],
        [1.0425],
        [1.2357],
        [0.9931],
        [1.0356],
        [1.0005],
        [0.8880],
        [1.0176],
        [0.9846],
        [1.0887],
        [1.1442],
        [1.1622],
        [0.9642],
        [0.8368],
        [1.1246],
        [0.9142],
        [1.0322],
        [0.8494],
        [1.0675],
        [0.7929],
        [0.9850],
        [1.2123],
        [1.2252],
        [0.8474],
        [1.1432],
        [1.0547],
        [1.0173],
        [0.9219],
        [1.0825],
        [1.0251],
        [1.0373],
        [0.9999],
        [0.8977],
        [1.2075],
        [1.2163],
        [1.0891],
        [0.8424],
        [0.9771],
        [0.8686],
        [1.1173],
        [1.0217],
        [1.0597],
        [1.0992],
       

Epoch number 0
 Current loss 0.5452011823654175

inputs are
torch.Size([200, 64])
torch.Size([200, 64])
OUTPUT
tensor([[1.0373],
        [0.8032],
        [0.9157],
        [1.1456],
        [1.0416],
        [0.9839],
        [0.8752],
        [1.1456],
        [0.9890],
        [0.9422],
        [1.1456],
        [0.8703],
        [0.8238],
        [0.9605],
        [0.9333],
        [0.9226],
        [0.9075],
        [0.8390],
        [0.8183],
        [0.9071],
        [0.9941],
        [0.9347],
        [1.1976],
        [1.0055],
        [0.8547],
        [0.8314],
        [1.0075],
        [1.1493],
        [1.0109],
        [1.1444],
        [0.7941],
        [1.0326],
        [0.9162],
        [1.0087],
        [0.8018],
        [1.0865],
        [0.8203],
        [0.8703],
        [0.8589],
        [0.9435],
        [0.7736],
        [1.0326],
        [0.8702],
        [1.0226],
        [0.9629],
        [0.8632],
        [0.6763],
        [0.8342],
        [1.0150],
       

Epoch number 0
 Current loss 0.47686469554901123

inputs are
torch.Size([200, 64])
torch.Size([200, 64])
OUTPUT
tensor([[1.0246],
        [1.0485],
        [0.9686],
        [1.2129],
        [0.8047],
        [1.1212],
        [0.8999],
        [1.0114],
        [0.9356],
        [1.0837],
        [0.8231],
        [1.1191],
        [1.0228],
        [0.9350],
        [1.0488],
        [1.1536],
        [1.0671],
        [1.1544],
        [1.0395],
        [0.9105],
        [1.0063],
        [0.9383],
        [0.9675],
        [0.8752],
        [1.1339],
        [1.0103],
        [0.9841],
        [1.0248],
        [1.1274],
        [1.1286],
        [1.0940],
        [1.0217],
        [1.0063],
        [0.9631],
        [0.8895],
        [1.2184],
        [1.0586],
        [1.0241],
        [0.8329],
        [0.9198],
        [1.1243],
        [1.1037],
        [1.1201],
        [0.9616],
        [0.8184],
        [1.1313],
        [1.1178],
        [0.9582],
        [0.8179],
      

Epoch number 0
 Current loss 0.6027752161026001

inputs are
torch.Size([200, 64])
torch.Size([200, 64])
OUTPUT
tensor([[1.0130],
        [1.1396],
        [1.2541],
        [1.0997],
        [1.0449],
        [0.9866],
        [1.0865],
        [0.9875],
        [1.2381],
        [1.3201],
        [1.1638],
        [1.2690],
        [0.9089],
        [1.0837],
        [1.1438],
        [1.0327],
        [1.0174],
        [0.9503],
        [0.7661],
        [1.0507],
        [1.0596],
        [1.1978],
        [1.0612],
        [0.9126],
        [1.1602],
        [1.0380],
        [1.0397],
        [1.1458],
        [0.8561],
        [1.2139],
        [0.9896],
        [1.1205],
        [1.1632],
        [1.0984],
        [1.1438],
        [1.0927],
        [0.8839],
        [1.1602],
        [1.1395],
        [1.1036],
        [0.8854],
        [1.1074],
        [1.0360],
        [0.9492],
        [0.8649],
        [0.6101],
        [0.9924],
        [0.8447],
        [1.1481],
       

KeyboardInterrupt: 

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    batch_num = 0
    with torch.no_grad():
        for i, data in enumerate(iterator,0):
            img0, img1 , label = data
            output = model(img0,img1)
            print(output)
            accuracy = binary_accuracy(output, label)
            print(accuracy)
            print("*********")

In [92]:
evaluate(model, val_batch_it, criterion)


tensor([[[0.9185],
         [0.9187],
         [0.9170],
         [0.9189],
         [0.9106],
         [0.9188],
         [0.9092],
         [0.8995],
         [0.8916],
         [0.9031],
         [0.9194],
         [0.9069],
         [0.9171],
         [0.9066],
         [0.9158],
         [0.9192],
         [0.9131],
         [0.9097],
         [0.9165],
         [0.8739],
         [0.9172],
         [0.9167],
         [0.9177],
         [0.9173],
         [0.9176],
         [0.9158],
         [0.9020],
         [0.9132],
         [0.7335],
         [0.9157],
         [0.9154],
         [0.8896],
         [0.9159],
         [0.9177],
         [0.9068],
         [0.9103],
         [0.9181],
         [0.8871],
         [0.9093],
         [0.9139],
         [0.9161],
         [0.9128],
         [0.9160],
         [0.8978],
         [0.8889],
         [0.9132],
         [0.9145],
         [0.8390],
         [0.9104],
         [0.9141],
         [0.9147],
         [0.9195],
         [0.

RuntimeError: Expected object of type torch.FloatTensor but found type torch.LongTensor for argument #2 'other'